import warnings
warnings.filterwarnings('ignore')

In [27]:
task_set=[]
with open('task_setEDF.txt', 'r') as file:
    for line in file:
        line = line.strip()
        if line.startswith('['):
            line = line[1:]
        if line.endswith(']'):
            line = line[:-1]
        values = line.split(',')
        task_set.append([values[1][2:len(values[1])-1],float(values[2]),float(values[3]),int(values[4]),int(values[5])])
print(task_set)

[['T1', 0.0, 3.0, 20, 20], ['T2', 0.0, 2.0, 5, 5], ['T3', 0.0, 4.0, 10, 10]]


In [28]:
def schedulability(task_set):
    util=0;
    for task in task_set:
        util+=(task[2]/min(task[3],task[4]))
    return (util<=1)
        

In [29]:
import math
def hyperperiod(periods):
    lcm_ = periods[0]
    for i in range(1, len(periods)):
        lcm_ = math.lcm(lcm_, periods[i])
    return lcm_

In [30]:
import pandas as pd

In [31]:
df=pd.DataFrame(task_set,columns=["Task","release_time","ex_time","period","Deadline"])

In [32]:
df

,Task,release_time,ex_time,period,Deadline
0,T1,0.0,3.0,20,20
1,T2,0.0,2.0,5,5
2,T3,0.0,4.0,10,10


In [33]:
df['time_left'] = df['ex_time']
df['c_deadline'] = df['Deadline']

In [34]:
df

,Task,release_time,ex_time,period,Deadline,time_left,c_deadline
0,T1,0.0,3.0,20,20,3.0,20
1,T2,0.0,2.0,5,5,2.0,5
2,T3,0.0,4.0,10,10,4.0,10


In [35]:
tl = []

In [36]:
if schedulability(task_set):
    for i in range(hyperperiod(df.period)):
        r_task=df[df["release_time"]<=i]
        r_task=r_task[r_task["time_left"]>0]
        if len(r_task)>0:
            s_task=r_task.sort_values('c_deadline').index[0]
            df.loc[s_task,"time_left"]=df.loc[s_task]["time_left"]-1.0
            if i>0 and tl[-1]['Task']==df.loc[s_task]["Task"] and tl[-1]['end']==i :
                tl[-1].update({'end':i+1,'length':tl[-1]['length']+1})
            else:
                tl += [{'Task': df.loc[s_task]["Task"],'start': i,'end': i+1,'length': 1}]
        arrived=df[(i+1)%df['period']==0].index
        df.loc[arrived, 'time_left'] = df.loc[arrived, 'ex_time']
        df.loc[arrived, 'c_deadline'] = df.loc[arrived, 'Deadline']+i+1
        print("i=",i)
        print(df)
else:
    print("Not Schedulable")

i= 0
  Task  release_time  ex_time  period  Deadline  time_left  c_deadline
0   T1           0.0      3.0      20        20        3.0          20
1   T2           0.0      2.0       5         5        1.0           5
2   T3           0.0      4.0      10        10        4.0          10
i= 1
  Task  release_time  ex_time  period  Deadline  time_left  c_deadline
0   T1           0.0      3.0      20        20        3.0          20
1   T2           0.0      2.0       5         5        0.0           5
2   T3           0.0      4.0      10        10        4.0          10
i= 2
  Task  release_time  ex_time  period  Deadline  time_left  c_deadline
0   T1           0.0      3.0      20        20        3.0          20
1   T2           0.0      2.0       5         5        0.0           5
2   T3           0.0      4.0      10        10        3.0          10
i= 3
  Task  release_time  ex_time  period  Deadline  time_left  c_deadline
0   T1           0.0      3.0      20        20        3.

In [37]:
tl

[{'Task': 'T2', 'start': 0, 'end': 2, 'length': 2},
 {'Task': 'T3', 'start': 2, 'end': 5, 'length': 3},
 {'Task': 'T2', 'start': 5, 'end': 7, 'length': 2},
 {'Task': 'T3', 'start': 7, 'end': 8, 'length': 1},
 {'Task': 'T1', 'start': 8, 'end': 10, 'length': 2},
 {'Task': 'T2', 'start': 10, 'end': 12, 'length': 2},
 {'Task': 'T1', 'start': 12, 'end': 13, 'length': 1},
 {'Task': 'T3', 'start': 13, 'end': 15, 'length': 2},
 {'Task': 'T2', 'start': 15, 'end': 17, 'length': 2},
 {'Task': 'T3', 'start': 17, 'end': 19, 'length': 2}]

In [38]:
tl=pd.DataFrame(tl)

In [39]:
tl

,Task,start,end,length
0,T2,0,2,2
1,T3,2,5,3
2,T2,5,7,2
3,T3,7,8,1
4,T1,8,10,2
5,T2,10,12,2
6,T1,12,13,1
7,T3,13,15,2
8,T2,15,17,2
9,T3,17,19,2
